# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_df = pd.read_csv('../output_data/cities.csv')
weather_data_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,32.11,80,0,12.66,CL,1662515999
1,1,ponta do sol,32.6667,-17.1000,70.09,75,35,2.44,PT,1662515992
2,2,bluff,-46.6000,168.3333,41.81,71,67,12.44,NZ,1662515985
3,3,castro,-24.7911,-50.0119,53.51,98,100,9.71,BR,1662516063
4,4,barrow,71.2906,-156.7887,48.22,71,0,4.61,US,1662516094
...,...,...,...,...,...,...,...,...,...,...
546,546,russell,32.3502,-85.1999,76.62,94,0,0.00,US,1662516396
547,547,lilongwe,-13.9669,33.7873,63.45,46,0,4.63,MW,1662516396
548,548,mehamn,71.0357,27.8492,40.95,93,100,3.44,NO,1662516396
549,549,souillac,-20.5167,57.5167,67.57,80,67,9.95,MU,1662516397


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data_df[["Latitude", "Longitude"]]
humidity = weather_data_df["Humidity"]
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

In [5]:
# Add Heatmap layer to map.
fig = gmaps.figure(map_type="HYBRID", center=(35.0, -80.0), zoom_level=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
weather_criteria_df = weather_data_df.loc[(weather_data_df['Max Temperature'] < 80) &
                        (weather_data_df['Max Temperature'] > 70) &
                        (weather_data_df['Wind Speed'] < 10) &
                        (weather_data_df['Cloudiness'] == 0)].dropna()
weather_criteria_df
                        

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
57,57,shu,43.5983,73.7614,75.24,19,0,8.57,KZ,1662516225
82,82,turayf,31.6725,38.6637,72.50,50,0,8.55,SA,1662516231
114,114,lompoc,34.6391,-120.4579,77.52,66,0,9.22,US,1662516137
118,118,weihai,37.5017,122.1136,74.26,62,0,9.28,CN,1662516240
132,132,russellville,35.2784,-93.1338,74.25,92,0,0.00,US,1662516245
191,191,rincon,32.2960,-81.2354,79.16,87,0,3.44,US,1662516267
229,229,tete,-16.1564,33.5867,70.38,68,0,7.65,MZ,1662516249
242,242,saint anthony,45.0205,-93.2180,72.90,78,0,0.00,US,1662516284
247,247,tucumcari,35.1717,-103.7250,78.64,34,0,4.61,US,1662516286
271,271,ajdabiya,30.7554,20.2263,73.74,70,0,7.99,LY,1662516294


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# hotel_df = weather_criteria_df
# hotel_df

In [8]:
hotel_df = weather_criteria_df
hotel_df["Hotel Name"] = ""

hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
57,57,shu,43.5983,73.7614,75.24,19,0,8.57,KZ,1662516225,
82,82,turayf,31.6725,38.6637,72.50,50,0,8.55,SA,1662516231,
114,114,lompoc,34.6391,-120.4579,77.52,66,0,9.22,US,1662516137,
118,118,weihai,37.5017,122.1136,74.26,62,0,9.28,CN,1662516240,
132,132,russellville,35.2784,-93.1338,74.25,92,0,0.00,US,1662516245,
191,191,rincon,32.2960,-81.2354,79.16,87,0,3.44,US,1662516267,
229,229,tete,-16.1564,33.5867,70.38,68,0,7.65,MZ,1662516249,
242,242,saint anthony,45.0205,-93.2180,72.90,78,0,0.00,US,1662516284,
247,247,tucumcari,35.1717,-103.7250,78.64,34,0,4.61,US,1662516286,
271,271,ajdabiya,30.7554,20.2263,73.74,70,0,7.99,LY,1662516294,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000, 
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {row['City']} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 57: shu.
Missing field/result... skipping.
------------
Retrieving Results for Index 82: turayf.
Missing field/result... skipping.
------------
Retrieving Results for Index 114: lompoc.
Missing field/result... skipping.
------------
Retrieving Results for Index 118: weihai.
Missing field/result... skipping.
------------
Retrieving Results for Index 132: russellville.
Missing field/result... skipping.
------------
Retrieving Results for Index 191: rincon.
Missing field/result... skipping.
------------
Retrieving Results for Index 229: tete.
Missing field/result... skipping.
------------
Retrieving Results for Index 242: saint anthony.
Missing field/result... skipping.
------------
Retrieving Results for Index 247: tucumcari.
Missing field/result... skipping.
------------
Retrieving Results for Index 271: ajdabiya.
Missing field/result... skipping.
------------
Retrieving Results for Index 273: jalu.
Missing field/result... skipping.
------------
Retrieving R

In [10]:
column_names = ["Hotel Name", "City", "Country",  "Latitude", "Longitude", "Max Temp", 
                "Humidity", "Cloudiness", "Wind Speed"]
hotel_df = hotel_df.reindex(columns=column_names)
hotel_df.head(10)

,Hotel Name,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
57,,shu,KZ,43.5983,73.7614,NaN,19,0,8.57
82,,turayf,SA,31.6725,38.6637,NaN,50,0,8.55
114,,lompoc,US,34.6391,-120.4579,NaN,66,0,9.22
118,,weihai,CN,37.5017,122.1136,NaN,62,0,9.28
132,,russellville,US,35.2784,-93.1338,NaN,92,0,0.00
191,,rincon,US,32.2960,-81.2354,NaN,87,0,3.44
229,,tete,MZ,-16.1564,33.5867,NaN,68,0,7.65
242,,saint anthony,US,45.0205,-93.2180,NaN,78,0,0.00
247,,tucumcari,US,35.1717,-103.7250,NaN,34,0,4.61
271,,ajdabiya,LY,30.7554,20.2263,NaN,70,0,7.99


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
# Create a combined map
heat_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)

fig

# Display figure

Figure(layout=FigureLayout(height='420px'))